In [ ]:
# --- CELL 1: GLOBAL IMPORTS ---
pip install -r requirements.22import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import glob
import torch
import torchvision.transforms.functional as F
from PIL import Image
import time

# Ensure inline plotting
%matplotlib inline

print("Environment Setup Complete.")

Intensity Transformations, Lab Space, and Histogram Equalization (Questions 1, 2 & 3)

In [ ]:
# --- CELL 2: FUNCTIONS FOR Q1, Q2, Q3 ---

def apply_gamma(image_np, gamma):
    # Normalize to [0, 1] if needed
    img = image_np.astype(float) / 255.0 if image_np.max() > 1.0 else image_np
    return np.power(img, gamma)

def apply_contrast_stretching(image_np, r1=0.2, r2=0.8):
    img = image_np.astype(float) / 255.0 if image_np.max() > 1.0 else image_np
    s = np.zeros_like(img)
    s[img < r1] = 0.0
    s[img > r2] = 1.0
    mask = (img >= r1) & (img <= r2)
    s[mask] = (img[mask] - r1) / (r2 - r1)
    return s

def _srgb_to_linear(c):
    return np.where(c <= 0.04045, c / 12.92, ((c + 0.055) / 1.055) ** 2.4)

def _linear_to_srgb(c):
    return np.where(c <= 0.0031308, 12.92 * c, 1.055 * (np.maximum(c, 0) ** (1 / 2.4)) - 0.055)

def rgb_to_lab(rgb):
    rgb_linear = _srgb_to_linear(rgb)
    X = 0.4124564 * rgb_linear[..., 0] + 0.3575761 * rgb_linear[..., 1] + 0.1804375 * rgb_linear[..., 2]
    Y = 0.2126729 * rgb_linear[..., 0] + 0.7151522 * rgb_linear[..., 1] + 0.0721750 * rgb_linear[..., 2]
    Z = 0.0193339 * rgb_linear[..., 0] + 0.1191920 * rgb_linear[..., 1] + 0.9503041 * rgb_linear[..., 2]
    Xn, Yn, Zn = 0.95047, 1.0, 1.08883
    delta = 6 / 29
    def f(t): return np.where(t > delta ** 3, np.cbrt(t), (t / (3 * delta ** 2)) + (4 / 29))
    fx, fy, fz = f(X/Xn), f(Y/Yn), f(Z/Zn)
    L = (116 * fy) - 16
    a = 500 * (fx - fy)
    b = 200 * (fy - fz)
    return np.stack([L, a, b], axis=-1)

def lab_to_rgb(lab):
    L, a, b = lab[..., 0], lab[..., 1], lab[..., 2]
    fy = (L + 16) / 116
    fx, fz = fy + (a / 500), fy - (b / 200)
    delta = 6 / 29
    def f_inv(t): return np.where(t > delta, t ** 3, 3 * (delta ** 2) * (t - (4 / 29)))
    Xn, Yn, Zn = 0.95047, 1.0, 1.08883
    X, Y, Z = Xn * f_inv(fx), Yn * f_inv(fy), Zn * f_inv(fz)
    r_lin = 3.2404542 * X - 1.5371385 * Y - 0.4985314 * Z
    g_lin = -0.9692660 * X + 1.8760108 * Y + 0.0415560 * Z
    b_lin = 0.0556434 * X - 0.2040259 * Y + 1.0572252 * Z
    return np.clip(_linear_to_srgb(np.stack([r_lin, g_lin, b_lin], axis=-1)), 0, 1)

def manual_hist_equalization(gray):
    gray_uint8 = (np.clip(gray, 0, 1) * 255).astype(np.uint8)
    hist = np.bincount(gray_uint8.flatten(), minlength=256)
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_scaled = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf_final = np.ma.filled(cdf_scaled, 0).astype(np.uint8)
    return cdf_final[gray_uint8].astype(float) / 255.0

Grayscale, Otsu and Masked Equalization (Question 4)

In [ ]:
# --- CELL 3: FUNCTIONS FOR Q4 ---

def otsu_threshold(image):
    histogram, _ = np.histogram(image.flatten(), bins=256, range=[0, 256])
    total = image.size
    sum_total = np.sum(np.arange(256) * histogram)
    sum_b, weight_b, max_var, threshold = 0, 0, 0, 0
    for t in range(256):
        weight_b += histogram[t]
        if weight_b == 0: continue
        weight_f = total - weight_b
        if weight_f == 0: break
        sum_b += t * histogram[t]
        m_b, m_f = sum_b / weight_b, (sum_total - sum_b) / weight_f
        var = weight_b * weight_f * (m_b - m_f) ** 2
        if var > max_var:
            max_var, threshold = var, t
    return threshold

def histogram_equalization_masked(image, mask):
    res = image.copy()
    masked_pixels = image[mask == 1]
    if len(masked_pixels) == 0: return res
    hist, _ = np.histogram(masked_pixels, bins=256, range=[0, 256])
    cdf = hist.cumsum()
    cdf_min = cdf[cdf > 0].min()
    cdf_norm = ((cdf - cdf_min) * 255 / (masked_pixels.size - cdf_min)).astype(np.uint8)
    res[mask == 1] = cdf_norm[image[mask == 1]]
    return res

In [ ]:
Gaussian Filtering and Derivatives (Questions 5 & 6)

In [ ]:
# --- CELL 4: FUNCTIONS FOR Q5, Q6 ---

def compute_gaussian_kernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x-(size//2))**2 + (y-(size//2))**2)/(2*sigma**2)), (size, size))
    return kernel / np.sum(kernel)

def compute_dog_kernel(size, sigma, direction='x'):
    center = size // 2
    y, x = np.ogrid[-center:center+1, -center:center+1]
    g = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    if direction == 'x':
        kernel = -(x / (sigma**2)) * g
    else:
        kernel = -(y / (sigma**2)) * g
    return kernel / np.sum(np.abs(kernel))

def visualize_kernel_3d(kernel, title="3D Kernel"):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    x, y = np.meshgrid(np.arange(kernel.shape[0]), np.arange(kernel.shape[1]))
    ax.plot_surface(x, y, kernel, cmap='viridis')
    plt.title(title)
    plt.show()

Zoom, Noise, Sharpening, and Bilateral (Questions 7, 8, 9 & 10)

In [ ]:
# --- CELL 5: FUNCTIONS FOR Q7 - Q10 ---

def zoom_bilinear(image, scale):
    h, w = image.shape[:2]
    nh, nw = int(h * scale), int(w * scale)
    res = np.zeros((nh, nw, image.shape[2]) if len(image.shape)==3 else (nh, nw), dtype=image.dtype)
    for i in range(nh):
        for j in range(nw):
            si, sj = i/scale, j/scale
            i1, j1 = int(si), int(sj)
            i2, j2 = min(i1+1, h-1), min(j1+1, w-1)
            di, dj = si-i1, sj-j1
            res[i,j] = (1-di)*(1-dj)*image[i1,j1] + (1-di)*dj*image[i1,j2] + di*(1-dj)*image[i2,j1] + di*dj*image[i2,j2]
    return res

def unsharp_masking(image, sigma=1.0, strength=1.5):
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    detail = image.astype(float) - blurred.astype(float)
    sharpened = np.clip(image.astype(float) + strength * detail, 0, 255).astype(np.uint8)
    return sharpened

def bilateral_filter_manual(image, d, sigma_s, sigma_r):
    h, w = image.shape
    res = np.zeros_like(image, dtype=float)
    radius = d // 2
    for i in range(h):
        for j in range(w):
            w_sum, pix_sum = 0, 0
            for ki in range(-radius, radius + 1):
                for kj in range(-radius, radius + 1):
                    ni, nj = np.clip(i+ki, 0, h-1), np.clip(j+kj, 0, w-1)
                    spatial_dist = ki**2 + kj**2
                    intensity_diff = (float(image[ni, nj]) - float(image[i, j]))**2
                    weight = np.exp(-spatial_dist/(2*sigma_s**2)) * np.exp(-intensity_diff/(2*sigma_r**2))
                    pix_sum += image[ni, nj] * weight
                    w_sum += weight
            res[i, j] = pix_sum / w_sum
    return res.astype(np.uint8)

Execution Block

In [ ]:
# --- CELL 6: DEMONSTRATION ---

# 1. Load Runway Image
runway = cv2.imread('Sources/runway.png', 0)
if runway is not None:
    # Q1 & Q3 Demo
    g05 = apply_gamma(runway, 0.5)
    stretched = apply_contrast_stretching(runway)
    equalized = manual_hist_equalization(runway / 255.0)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(131); plt.imshow(g05, cmap='gray'); plt.title("Gamma 0.5")
    plt.subplot(132); plt.imshow(stretched, cmap='gray'); plt.title("Contrast Stretch")
    plt.subplot(133); plt.imshow(equalized, cmap='gray'); plt.title("Hist Equalized")
    plt.show()

# 2. Q5 Gaussian Kernel Demo
k51 = compute_gaussian_kernel(51, 2)
visualize_kernel_3d(k51, "51x51 Gaussian Kernel (Sigma=2)")

# 3. Q6 DoG Demo
dog_k = compute_dog_kernel(51, 2, 'x')
visualize_kernel_3d(dog_k, "Derivative of Gaussian (X)")

print("Processing complete. Check 'Results' folder for saved outputs if your scripts use imwrite.")